In [1]:
import os

import pandas as pd
import math
import random as rd
import numpy as np
import datetime as dt
import tensorflow as tf
from collections import deque
import pickle
import dlib

from deepface import DeepFace
from deepface.basemodels import VGGFace, OpenFace, Facenet, FbDeepFace, DeepID
from deepface.extendedmodels import Age, Gender, Race, Emotion
#from deepface.modules import verification
from deepface.commons import functions, distance as dst

from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Input,Conv2D,MaxPooling2D,Dropout,LSTM,\
                            TimeDistributed,Flatten,Dense,Bidirectional,ConvLSTM2D,MaxPooling3D,AveragePooling2D,Lambda,\
                            Activation,BatchNormalization

from tensorflow import keras
tf.test.gpu_device_name()

2024-08-10 22:08:24.551298: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


''

In [2]:
seed_constant = 27
np.random.seed(seed_constant)
rd.seed(seed_constant)
tf.random.set_seed(seed_constant)

In [3]:
DIR = '/var/scratch/mpa326/'

In [4]:
dbfile = open('/var/scratch/mpa326/CREMA-D_code/Video Features/video_train_50m_50f', 'rb')     
df = pickle.load(dbfile)

y_train_50m_50f = df['labels']
gender_train_50m_50f = df['genders']
X_train_50m_50f=df['features']
race_train_50m_50f = df['races']
ethnicity_train_50m_50f = df['ethnicity']
paths_train_50m_50f = df['video_files_paths']

In [5]:
dbfile = open('/var/scratch/mpa326/CREMA-D_code/Video Features/video_train_40m_60f', 'rb')     
df = pickle.load(dbfile)

y_train_40m_60f = df['labels']
gender_train_40m_60f = df['genders']
X_train_40m_60f=df['features']
race_train_40m_60f = df['races']
ethnicity_train_40m_60f = df['ethnicity']
paths_train_40m_60f = df['video_files_paths']

In [6]:
dbfile = open('/var/scratch/mpa326/CREMA-D_code/Video Features/video_train_60m_40f', 'rb')     
df = pickle.load(dbfile)

y_train_60m_40f = df['labels']
gender_train_60m_40f = df['genders']
X_train_60m_40f=df['features']
race_train_60m_40f = df['races']
ethnicity_train_60m_40f = df['ethnicity']
paths_train_60m_40f = df['video_files_paths']

In [7]:
dbfile = open('/var/scratch/mpa326/CREMA-D_code/Video Features/video_test_common', 'rb')     
df = pickle.load(dbfile)

y_test = df['labels']
gender_test = df['genders']
X_test=df['features']
race_test = df['races']
ethnicity_test = df['ethnicity']
paths_test = df['video_files_paths']

In [8]:
dbfile = open('/var/scratch/mpa326/CREMA-D_code/Audio Features/audio_train_50m_50f', 'rb')     
df = pickle.load(dbfile)
X_train_aud_50m_50f=df['features']

In [9]:
dbfile = open('/var/scratch/mpa326/CREMA-D_code/Audio Features/audio_train_40m_60f', 'rb')     
df = pickle.load(dbfile)
X_train_aud_40m_60f=df['features']

In [10]:
dbfile = open('/var/scratch/mpa326/CREMA-D_code/Audio Features/audio_train_60m_40f', 'rb')     
df = pickle.load(dbfile)
X_train_aud_60m_40f=df['features']

In [11]:
dbfile = open('/var/scratch/mpa326/CREMA-D_code/Audio Features/audio_test_common', 'rb')     
df = pickle.load(dbfile)
X_test_aud=df['features']

In [12]:
from sklearn.utils import shuffle
X_train_50m_50f, X_train_aud_50m_50f, y_train_50m_50f, gender_train_50m_50f, race_train_50m_50f, ethnicity_train_50m_50f = shuffle(X_train_50m_50f, X_train_aud_50m_50f, y_train_50m_50f, gender_train_50m_50f, race_train_50m_50f, ethnicity_train_50m_50f, random_state=27)
X_train_50m_50f, X_train_aud_50m_50f, y_train_50m_50f, gender_train_50m_50f, race_train_50m_50f, ethnicity_train_50m_50f = shuffle(X_train_50m_50f, X_train_aud_50m_50f, y_train_50m_50f, gender_train_50m_50f, race_train_50m_50f, ethnicity_train_50m_50f, random_state=7)
X_train_50m_50f, X_train_aud_50m_50f, y_train_50m_50f, gender_train_50m_50f, race_train_50m_50f, ethnicity_train_50m_50f = shuffle(X_train_50m_50f, X_train_aud_50m_50f, y_train_50m_50f, gender_train_50m_50f, race_train_50m_50f, ethnicity_train_50m_50f, random_state=42)

In [13]:
X_train_40m_60f, X_train_aud_40m_60f, y_train_40m_60f, gender_train_40m_60f, race_train_40m_60f, ethnicity_train_40m_60f = shuffle(X_train_40m_60f, X_train_aud_40m_60f, y_train_40m_60f, gender_train_40m_60f, race_train_40m_60f, ethnicity_train_40m_60f, random_state=27)
X_train_40m_60f, X_train_aud_40m_60f, y_train_40m_60f, gender_train_40m_60f, race_train_40m_60f, ethnicity_train_40m_60f = shuffle(X_train_40m_60f, X_train_aud_40m_60f, y_train_40m_60f, gender_train_40m_60f, race_train_40m_60f, ethnicity_train_40m_60f, random_state=7)
X_train_40m_60f, X_train_aud_40m_60f, y_train_40m_60f, gender_train_40m_60f, race_train_40m_60f, ethnicity_train_40m_60f = shuffle(X_train_40m_60f, X_train_aud_40m_60f, y_train_40m_60f, gender_train_40m_60f, race_train_40m_60f, ethnicity_train_40m_60f, random_state=42)

In [14]:
X_train_60m_40f, X_train_aud_60m_40f, y_train_60m_40f, gender_train_60m_40f, race_train_60m_40f, ethnicity_train_60m_40f = shuffle(X_train_60m_40f, X_train_aud_60m_40f, y_train_60m_40f, gender_train_60m_40f, race_train_60m_40f, ethnicity_train_60m_40f, random_state=27)
X_train_60m_40f, X_train_aud_60m_40f, y_train_60m_40f, gender_train_60m_40f, race_train_60m_40f, ethnicity_train_60m_40f = shuffle(X_train_60m_40f, X_train_aud_60m_40f, y_train_60m_40f, gender_train_60m_40f, race_train_60m_40f, ethnicity_train_60m_40f, random_state=7)
X_train_60m_40f, X_train_aud_60m_40f, y_train_60m_40f, gender_train_60m_40f, race_train_60m_40f, ethnicity_train_60m_40f = shuffle(X_train_60m_40f, X_train_aud_60m_40f, y_train_60m_40f, gender_train_60m_40f, race_train_60m_40f, ethnicity_train_60m_40f, random_state=42)

In [15]:
X_test, X_test_aud, y_test, gender_test, race_test, ethnicity_test = shuffle(X_test, X_test_aud, y_test, gender_test, race_test, ethnicity_test, random_state=27)
X_test, X_test_aud, y_test, gender_test, race_test, ethnicity_test = shuffle(X_test, X_test_aud, y_test, gender_test, race_test, ethnicity_test, random_state=7)
X_test, X_test_aud, y_test, gender_test, race_test, ethnicity_test = shuffle(X_test, X_test_aud, y_test, gender_test, race_test, ethnicity_test, random_state=42)

In [16]:
from tensorflow.keras.models import Model
emotion=Emotion.loadModel()
for layer in emotion.layers:
    print(layer.name)
features_emotion=Model(inputs=emotion.input, outputs=emotion.get_layer('dense_1').output)

conv2d
max_pooling2d
conv2d_1
conv2d_2
average_pooling2d
conv2d_3
conv2d_4
average_pooling2d_1
flatten
dense
dropout
dense_1
dropout_1
dense_2


In [17]:
df_ = np.ones((4678,20,48,48))
for i in range(4678):
  for j in range(20):
    img=X_train_50m_50f[i][j]
    img=img.astype(np.float32)
    # df_[i][j]=cv2.resize(img,(48, 48))/255
    df_[i][j]=img/255
X_train_50m_50f=df_
df_=[]

In [18]:
df_ = np.ones((4678,20,48,48))
for i in range(4678):
  for j in range(20):
    img=X_train_40m_60f[i][j]
    img=img.astype(np.float32)
    # df_[i][j]=cv2.resize(img,(48, 48))/255
    df_[i][j]=img/255
X_train_40m_60f=df_
df_=[]

In [19]:
df_ = np.ones((4677,20,48,48))
for i in range(4677):
  for j in range(20):
    img=X_train_60m_40f[i][j]
    img=img.astype(np.float32)
    # df_[i][j]=cv2.resize(img,(48, 48))/255
    df_[i][j]=img/255
X_train_60m_40f=df_
df_=[]

In [20]:
df_ = np.ones((1487,20,48,48))
for i in range(1487):
  for j in range(20):
    img=X_test[i][j]
    img=img.astype(np.float32)
    # df_[i][j]=cv2.resize(img,(48, 48))/255
    df_[i][j]=img/255
X_test=df_
df_=[]

In [21]:
features_emotion.trainable=False

In [22]:
class Localization(tf.keras.layers.Layer):
    def __init__(self):
        super(Localization, self).__init__()
        self.pool1 = tf.keras.layers.MaxPool2D()
        self.conv1 = tf.keras.layers.Conv2D(20, [5, 5], activation='relu')
        self.pool2 = tf.keras.layers.MaxPool2D()
        self.conv2 = tf.keras.layers.Conv2D(20, [5, 5], activation='relu')
        self.flatten = tf.keras.layers.Flatten()
        self.fc1 = tf.keras.layers.Dense(20, activation='relu')
        self.fc2 = tf.keras.layers.Dense(6, activation=None, bias_initializer=tf.keras.initializers.constant([1.0, 0.0, 0.0, 0.0, 1.0, 0.0]), kernel_initializer='zeros')

    def build(self, input_shape):
        print("Building Localization Network with input shape:", input_shape)

    def compute_output_shape(self, input_shape):
        return [None, 2, 3]

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        theta = self.fc2(x)
        theta = tf.keras.layers.Reshape((2, 3))(theta)
        return theta

In [23]:
class BilinearInterpolation(tf.keras.layers.Layer):
    def __init__(self, height=40, width=40):
        super(BilinearInterpolation, self).__init__()
        self.height = height
        self.width = width

    def compute_output_shape(self, input_shape):
        return [None, self.height, self.width, 1]

    def get_config(self):
        return {
            'height': self.height,
            'width': self.width,
        }
    
    def build(self, input_shape):
        print("Building Bilinear Interpolation Layer with input shape:", input_shape)

    def advance_indexing(self, inputs, x, y):
        '''
        Numpy like advance indexing is not supported in tensorflow, hence, this function is a hack around the same method
        '''        
        shape = tf.shape(inputs)
        batch_size, _, _ = shape[0], shape[1], shape[2]
        
        batch_idx = tf.range(0, batch_size)
        batch_idx = tf.reshape(batch_idx, (batch_size, 1, 1))
        b = tf.tile(batch_idx, (1, self.height, self.width))
        indices = tf.stack([b, y, x], 3)
        return tf.gather_nd(inputs, indices)

    def call(self, inputs):
        images, theta = inputs
        homogenous_coordinates = self.grid_generator(batch=tf.shape(images)[0])
        return self.interpolate(images, homogenous_coordinates, theta)

    def grid_generator(self, batch):
        x = tf.linspace(-1, 1, self.width)
        y = tf.linspace(-1, 1, self.height)
            
        xx, yy = tf.meshgrid(x, y)
        xx = tf.reshape(xx, (-1,))
        yy = tf.reshape(yy, (-1,))
        homogenous_coordinates = tf.stack([xx, yy, tf.ones_like(xx)])
        homogenous_coordinates = tf.expand_dims(homogenous_coordinates, axis=0)
        homogenous_coordinates = tf.tile(homogenous_coordinates, [batch, 1, 1])
        homogenous_coordinates = tf.cast(homogenous_coordinates, dtype=tf.float32)
        return homogenous_coordinates
    
    def interpolate(self, images, homogenous_coordinates, theta):

        with tf.name_scope("Transformation"):
            transformed = tf.matmul(theta, homogenous_coordinates)
            transformed = tf.transpose(transformed, perm=[0, 2, 1])
            transformed = tf.reshape(transformed, [-1, self.height, self.width, 2])
                
            x_transformed = transformed[:, :, :, 0]
            y_transformed = transformed[:, :, :, 1]
                
            x = ((x_transformed + 1.) * tf.cast(self.width, dtype=tf.float32)) * 0.5
            y = ((y_transformed + 1.) * tf.cast(self.height, dtype=tf.float32)) * 0.5

        with tf.name_scope("VariableCasting"):
            x0 = tf.cast(tf.math.floor(x), dtype=tf.int32)
            x1 = x0 + 1
            y0 = tf.cast(tf.math.floor(y), dtype=tf.int32)
            y1 = y0 + 1

            x0 = tf.clip_by_value(x0, 0, self.width-1)
            x1 = tf.clip_by_value(x1, 0, self.width-1)
            y0 = tf.clip_by_value(y0, 0, self.height-1)
            y1 = tf.clip_by_value(y1, 0, self.height-1)
            x = tf.clip_by_value(x, 0, tf.cast(self.width, dtype=tf.float32)-1.0)
            y = tf.clip_by_value(y, 0, tf.cast(self.height, dtype=tf.float32)-1)

        with tf.name_scope("AdvanceIndexing"):
            Ia = self.advance_indexing(images, x0, y0)
            Ib = self.advance_indexing(images, x0, y1)
            Ic = self.advance_indexing(images, x1, y0)
            Id = self.advance_indexing(images, x1, y1)

        with tf.name_scope("Interpolation"):
            x0 = tf.cast(x0, dtype=tf.float32)
            x1 = tf.cast(x1, dtype=tf.float32)
            y0 = tf.cast(y0, dtype=tf.float32)
            y1 = tf.cast(y1, dtype=tf.float32)
                            
            wa = (x1-x) * (y1-y)
            wb = (x1-x) * (y-y0)
            wc = (x-x0) * (y1-y)
            wd = (x-x0) * (y-y0)

            wa = tf.expand_dims(wa, axis=3)
            wb = tf.expand_dims(wb, axis=3)
            wc = tf.expand_dims(wc, axis=3)
            wd = tf.expand_dims(wd, axis=3)
                        
        return tf.math.add_n([wa*Ia + wb*Ib + wc*Ic + wd*Id])

In [24]:
emotions_en = ['ANG', 'DIS', 'FEA', 'HAP', 'NEU', 'SAD']

In [25]:
def create__LRCN_with_STN() :
  image = tf.keras.layers.Input(shape=(20,48,48,1))
  theta = TimeDistributed(Localization())(image)
  x = TimeDistributed(BilinearInterpolation(height=48, width=48))([image, theta])
  features=TimeDistributed(features_emotion)(x)
  lstm=Bidirectional(LSTM(200, activation='tanh',input_shape=(20, 1024),dropout=.3))(features)
  out=Dense(len(emotions_en), activation = 'softmax')(lstm)

  return tf.keras.models.Model(inputs=image, outputs=out)


In [26]:
def get_model_name(k):
    return 'model_'+str(k)+'.h5'

In [27]:
from sklearn.model_selection import StratifiedKFold

seed = 7
kfold = StratifiedKFold(n_splits=5, shuffle=False, random_state=None)

In [28]:
unique_values, counts = np.unique(gender_train_50m_50f, return_counts=True)

# Print the results
for value, count in zip(unique_values, counts):
    print(f"Value {value} appears {count} times")


Value Female appears 2338 times
Value Male appears 2340 times


In [29]:
unique_values, counts = np.unique(race_train_50m_50f, return_counts=True)

# Print the results
for value, count in zip(unique_values, counts):
    print(f"Value {value} appears {count} times")


Value African American appears 1139 times
Value Asian appears 346 times
Value Caucasian appears 3140 times
Value Unknown appears 53 times


In [30]:
unique_values, counts = np.unique(gender_train_60m_40f, return_counts=True)

# Print the results
for value, count in zip(unique_values, counts):
    print(f"Value {value} appears {count} times")


Value Female appears 1870 times
Value Male appears 2807 times


In [31]:
unique_values, counts = np.unique(race_test, return_counts=True)

# Print the results
for value, count in zip(unique_values, counts):
    print(f"Value {value} appears {count} times")

Value African American appears 345 times
Value Asian appears 122 times
Value Caucasian appears 1006 times
Value Unknown appears 14 times


In [32]:
X_train_aud_50m_50f = X_train_aud_50m_50f/255
X_train_aud_40m_60f = X_train_aud_40m_60f/255
X_train_aud_60m_40f = X_train_aud_60m_40f/255
X_test_aud = X_test_aud/255

In [33]:
def model_alexnet():
  AlexNet = Sequential()

  #1st Convolutional Layer
  AlexNet.add(Conv2D(filters=96, input_shape=(128,128,3), kernel_size=(11,11), strides=(4,4), padding='same'))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))
  AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(1,1), padding='same'))

  #2nd Convolutional Layer
  AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1,1), padding='same'))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))
  AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(1,1), padding='same'))

  #3rd Convolutional Layer
  AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))

  #4th Convolutional Layer
  AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))

  #5th Convolutional Layer
  AlexNet.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))
  AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(1,1), padding='same'))

  #Passing it to a Fully Connected layer
  AlexNet.add(Flatten())
  AlexNet.add(Dense(4096, input_shape=(32,32,3,)))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))
  AlexNet.add(Dropout(0.4))

  # #2nd Fully Connected Layer
  # AlexNet.add(Dense(4096))
  # AlexNet.add(BatchNormalization())
  # AlexNet.add(Activation('relu'))
  # #Add Dropout
  # AlexNet.add(Dropout(0.4))

  #3rd Fully Connected Layer
  AlexNet.add(Dense(1000))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('relu'))
  #Add Dropout
  AlexNet.add(Dropout(0.4))

  #Output Layer
  AlexNet.add(Dense(6))
  AlexNet.add(BatchNormalization())
  AlexNet.add(Activation('softmax'))

  return AlexNet 

In [ ]:
#LATE FUSION

In [34]:
video_indice = 3
audio_indice = 1

In [35]:
from tensorflow.keras.models import load_model

save_dir = os.path.abspath(DIR)+'/cremad_40m60f_audio_model/'
model_audio = model_alexnet()
model_audio.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=["accuracy"])
model_audio.load_weights(save_dir +"model_"+str(audio_indice)+".h5")

In [36]:
save_dir = os.path.abspath(DIR)+'/cremad_40m60f_video_model/'

model_video = create__LRCN_with_STN()
model_video.compile(loss='sparese_categorical_crossentropy', optimizer='Adam', metrics=["accuracy"])
model_video.load_weights(save_dir+"model_"+str(video_indice)+".h5")

Building Localization Network with input shape: (None, 48, 48, 1)
Building Bilinear Interpolation Layer with input shape: ((None, 48, 48, 1), (None, 2, 3))


In [37]:
features_video_model=Model(inputs=model_video.input, outputs=model_video.get_layer('bidirectional').output)

In [38]:
features_video=features_video_model.predict(X_train_40m_60f)

147/147 [==============================] - 28s 177ms/step


In [39]:
features_video.shape

(4678, 400)

In [40]:
features_audio_model=Model(inputs=model_audio.input, outputs=model_audio.get_layer('dropout_3').output)

In [41]:
features_audio=features_audio_model.predict(X_train_aud_40m_60f)

147/147 [==============================] - 171s 1s/step


In [42]:
features_audio.shape

(4678, 1000)

In [43]:
fused_data_train=np.concatenate((features_video,features_audio),axis=1)

In [44]:
print(fused_data_train.shape)
print(fused_data_train)

(4678, 1400)
[[ 0.27680057  0.40438     0.17499319 ...  0.8896747   0.33789122
   0.        ]
 [-0.01256334 -0.731337   -0.11927307 ...  0.10878398  0.66160756
   0.        ]
 [ 0.01654009  0.0363176  -0.19858484 ...  1.7530757   1.6893493
   0.        ]
 ...
 [-0.01857208 -0.8176755  -0.34945592 ...  0.          0.
   0.41210353]
 [ 0.02107946  0.28272215  0.28229865 ...  1.0430069   0.37827185
   0.        ]
 [ 0.00230955  0.0302401   0.06474552 ...  0.13205327  0.
   0.        ]]


In [45]:
features_video_test=features_video_model.predict(X_test)

47/47 [==============================] - 9s 182ms/step


In [46]:
features_audio_test=features_audio_model.predict(X_test_aud)

47/47 [==============================] - 62s 1s/step


In [47]:
fused_data_test = np.concatenate((features_video_test, features_audio_test), axis=1)

In [48]:
fused_data_test.shape

(1487, 1400)

In [49]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
fused_data_train = sc.fit_transform(fused_data_train)
fused_data_test = sc.transform(fused_data_test)

In [50]:
from sklearn.decomposition import PCA

def find_n_components(data, variance_threshold=0.95):
    """
    Determine the number of components to retain a specified amount of variance.

    Parameters:
    - data: array-like, shape (n_samples, n_features)
      The data to perform PCA on.
    - variance_threshold: float, optional (default=0.95)
      The amount of variance that needs to be retained. Should be between 0 and 1.

    Returns:
    - n_components: int
      The number of components to retain the specified amount of variance.
    """
    pca = PCA()
    pca.fit(data)
    
    # Calculate cumulative variance
    cumulative_variance = pca.explained_variance_ratio_.cumsum()
    
    # Find the number of components that satisfy the variance threshold
    n_components = (cumulative_variance >= variance_threshold).argmax() + 1
    
    return n_components

# Example usage:
# Assuming fused_data_train is your training dataset
n_components = find_n_components(fused_data_train, variance_threshold=0.95)
print(f"Number of components to retain 95% variance: {n_components}")

# Applying PCA with the determined number of components
lda = PCA(n_components=n_components)
fused_data_train = lda.fit_transform(fused_data_train)
fused_data_test = lda.transform(fused_data_test)

Number of components to retain 95% variance: 230


In [51]:
fused_data_train.shape

(4678, 230)

In [52]:
def multi_modal_NN():
  model=Sequential()
  model.add(Input(shape=(230)))
  
  model.add(Dense(2096))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0.4))

  model.add(Dense(1024))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0.4))

  model.add(Dense(6))
  model.add(BatchNormalization())
  model.add(Activation('softmax'))
  return model

In [111]:
VALIDATION_ACCURACY_SPEECH = []
VALIDATION_LOSS_SPEECH = []

save_dir = os.path.abspath(DIR) + '/cremad_40m60f_latefusion_PCA_n230_model/'
fold_var = 1

for train_idx, val_idx in kfold.split(fused_data_train, y_train_40m_60f):
    model=multi_modal_NN()

    model.compile(loss='sparse_categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])

    early_stopping_callback = EarlyStopping(monitor = 'val_accuracy', patience = 15, restore_best_weights = True)

    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(save_dir + get_model_name(fold_var), monitor='val_accuracy',save_best_only=True, mode='max')
# Start training the model.
    LRCN_model_training_history = model.fit(      x = fused_data_train[train_idx],
                                                  y = y_train_40m_60f[train_idx],
                                                  validation_data=(fused_data_train[val_idx], y_train_40m_60f[val_idx]),
                                                  epochs = 80,
                                                  batch_size = 4,
                                                  shuffle = True,
                                                  callbacks = [checkpoint_cb, early_stopping_callback])
    model.load_weights(save_dir + "model_" + str(fold_var) + ".h5")
	
    results = model.evaluate(fused_data_test, y_test)
    results = dict(zip(model.metrics_names, results))
	
    VALIDATION_ACCURACY_SPEECH.append(results['accuracy'])
    VALIDATION_LOSS_SPEECH.append(results['loss'])
	
    tf.keras.backend.clear_session()
	
    fold_var += 1
     

Epoch 1/80
936/936 [==============================] - 13s 12ms/step - loss: 0.9004 - accuracy: 0.7288 - val_loss: 0.9271 - val_accuracy: 0.6912
Epoch 2/80
936/936 [==============================] - 11s 11ms/step - loss: 0.7579 - accuracy: 0.7803 - val_loss: 0.8818 - val_accuracy: 0.6955
Epoch 3/80
936/936 [==============================] - 11s 11ms/step - loss: 0.6897 - accuracy: 0.8044 - val_loss: 0.9134 - val_accuracy: 0.6795
Epoch 4/80
936/936 [==============================] - 11s 12ms/step - loss: 0.6534 - accuracy: 0.8129 - val_loss: 0.8884 - val_accuracy: 0.6955
Epoch 5/80
936/936 [==============================] - 11s 12ms/step - loss: 0.6049 - accuracy: 0.8239 - val_loss: 0.9019 - val_accuracy: 0.7105
Epoch 6/80
936/936 [==============================] - 11s 12ms/step - loss: 0.5592 - accuracy: 0.8391 - val_loss: 0.9120 - val_accuracy: 0.7115
Epoch 7/80
936/936 [==============================] - 11s 12ms/step - loss: 0.5383 - accuracy: 0.8498 - val_loss: 0.9160 - val_accuracy:

In [53]:
#Fairness
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
save_dir = os.path.abspath(DIR)+'/cremad_40m60f_latefusion_PCA_n230_model/'
final_model = multi_modal_NN()
final_model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=["accuracy"])
final_model.load_weights(save_dir + "model_4.h5")  # Adjust to load the best model if necessary

test_results = final_model.evaluate(fused_data_test, y_test)
test_results = dict(zip(final_model.metrics_names, test_results))

print("Test set evaluation:", test_results)

# Gender bias analysis
metrics_by_gender = {}
for gender in np.unique(gender_test):
    gender_indices = np.where(gender_test == gender)
    X_gender_test = fused_data_test[gender_indices]
    y_gender_test = y_test[gender_indices]
    y_pred = np.argmax(final_model.predict(X_gender_test), axis=1)

    accuracy = np.mean(y_pred == y_gender_test)
    precision = precision_score(y_gender_test, y_pred, average='macro')
    recall = recall_score(y_gender_test, y_pred, average='macro')
    f1 = f1_score(y_gender_test, y_pred, average='macro')

    cm = confusion_matrix(y_gender_test, y_pred)
    tn = np.sum(cm) - (np.sum(cm, axis=0) + np.sum(cm, axis=1) - np.diag(cm))
    fp = np.sum(cm, axis=0) - np.diag(cm)
    fn = np.sum(cm, axis=1) - np.diag(cm)
    tp = np.diag(cm)
    tpr = tp / (tp + fn)  # True Positive Rate
    fpr = fp / (fp + tn)  # False Positive Rate
    fnr = fn / (fn + tp)  # False Negative Rate
    precision_value = tp / (tp + fp)

    demographic_parity_dict = {}
    total_count  = len(y_pred)
    for emotion_idx in np.unique(y_pred):
        #emotion_idx = int(emotion_idx)
        count_emotion = np.count_nonzero(y_pred == emotion_idx)
        demographic_parity = count_emotion / total_count 
        emotion_name = emotions_en[emotion_idx]  # Get the emotion name from the index
        demographic_parity_dict[emotion_name] = demographic_parity

    
    metrics_by_gender[gender] = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'tpr': tpr,
        'fpr_value': fpr,
        'fnr_value': fnr,
        'precision_value': precision_value,
        'demographic_parity': demographic_parity_dict
    }
    
    

for gender in metrics_by_gender:
    #metrics_by_gender[gender]['demographic_parity'] = metrics_by_gender[gender]['positive_rate']
    metrics_by_gender[gender]['equalized_odds'] = (metrics_by_gender[gender]['tpr'], metrics_by_gender[gender]['fpr_value'])
    metrics_by_gender[gender]['equal_opportunity'] = metrics_by_gender[gender]['tpr']
    metrics_by_gender[gender]['predictive_parity'] = metrics_by_gender[gender]['precision_value']



47/47 [==============================] - 1s 5ms/step - loss: 0.8574 - accuracy: 0.7256
Test set evaluation: {'loss': 0.8574368357658386, 'accuracy': 0.7256220579147339}
25/25 [==============================] - 0s 4ms/step


In [54]:
header = ['gender', 'accuracy', 'precision', 'recall', 'f1', 'tpr', 'fpr_value', 'fnr_value', 'precision_value', 'demographic_parity']
import csv
file_path = '/var/scratch/mpa326/CREMA-D_code/FairnessResults/fairness_analysis_cremad_40m60f_latefusion_model.csv'

# Ensure the directory exists
os.makedirs(os.path.dirname(file_path), exist_ok=True)
# Write the dictionary to a CSV file
with open(file_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(header)
    
    for gender, metrics in metrics_by_gender.items():
        row = [gender]
        for key in header[1:]:  # Exclude 'gender' from header keys
            value = metrics.get(key, None)
            # Convert dictionaries to string for CSV export
            if isinstance(value, dict):
                value = str(value)
            row.append(value)
        writer.writerow(row)
    overall_row = ['overall']
    for key in header[1:]:
        # Add overall accuracy and loss, leave other columns empty (None)
        if key in test_results:
            overall_row.append(test_results[key])
        else:
            overall_row.append(None)

In [ ]:
#Early Fusion

In [55]:
X_train_flattened = np.reshape(X_train_40m_60f, (4678, -1))

X_train_aud_flattened = np.reshape(X_train_aud_40m_60f, (4678, -1))
X_train_aud_flattened.shape

(4678, 49152)

In [56]:
X_test_flattened = np.reshape(X_test, (1487, -1))

X_test_aud_flattened = np.reshape(X_test_aud, (1487, -1))
X_test_aud_flattened.shape

(1487, 49152)

In [57]:
from sklearn.preprocessing import StandardScaler
video_scaler = StandardScaler().fit(X_train_flattened)
video_data_standardized = video_scaler.fit_transform(X_train_flattened)
video_data_standardized_test = video_scaler.fit_transform(X_test_flattened)

In [58]:
video_data_standardized_test.shape

(1487, 46080)

In [59]:
audio_scaler = StandardScaler().fit(X_train_aud_flattened)
audio_data_standardized = audio_scaler.fit_transform(X_train_aud_flattened)
audio_data_standardized_test = audio_scaler.fit_transform(X_test_aud_flattened)

In [60]:
audio_data_standardized_test.shape

(1487, 49152)

In [61]:
from sklearn.decomposition import PCA
pca_video = PCA(n_components=0.95).fit(video_data_standardized)
video_data_pca = pca_video.fit_transform(video_data_standardized)
video_data_pca_test = pca_video.transform(video_data_standardized_test)

In [62]:
video_data_pca.shape

(4678, 1365)

In [63]:
pca_audio = PCA(n_components=0.95).fit(audio_data_standardized)
audio_data_pca = pca_audio.fit_transform(audio_data_standardized)
audio_data_pca_test = pca_audio.transform(audio_data_standardized_test)

In [64]:
audio_data_pca_test.shape

(1487, 720)

In [65]:
concatenated_train_data = np.concatenate((video_data_pca, audio_data_pca), axis=1)

In [66]:
concatenated_train_data.shape

(4678, 2085)

In [67]:
concatenated_test_data = np.concatenate((video_data_pca_test, audio_data_pca_test), axis=1)

In [68]:
concatenated_test_data.shape

(1487, 2085)

In [69]:
def multi_modal_NN():
  model=Sequential()
  model.add(Input(shape=(2085)))
  
  model.add(Dense(2096))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0.4))

  model.add(Dense(1024))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0.4))

  model.add(Dense(8))
  model.add(BatchNormalization())
  model.add(Activation('softmax'))
  return model

In [113]:
VALIDATION_ACCURACY_SPEECH = []
VALIDATION_LOSS_SPEECH = []

save_dir = os.path.abspath(DIR) + '/cremad_40m60f_earlyfusion_PCA_n0.95_model/'
fold_var = 1

for train_idx, val_idx in kfold.split(concatenated_train_data, y_train_40m_60f):
    model=multi_modal_NN()

    model.compile(loss='sparse_categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])

    early_stopping_callback = EarlyStopping(monitor = 'val_accuracy', patience = 15, restore_best_weights = True)

    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(save_dir + get_model_name(fold_var), monitor='val_accuracy',save_best_only=True, mode='max')
# Start training the model.
    LRCN_model_training_history = model.fit(      x = concatenated_train_data[train_idx],
                                                  y = y_train_40m_60f[train_idx],
                                                  validation_data=(concatenated_train_data[val_idx], y_train_40m_60f[val_idx]),
                                                  epochs = 80,
                                                  batch_size = 4,
                                                  shuffle = True,
                                                  callbacks = [checkpoint_cb, early_stopping_callback])
    model.load_weights(save_dir + "model_" + str(fold_var) + ".h5")
	
    results = model.evaluate(concatenated_test_data, y_test)
    results = dict(zip(model.metrics_names, results))
	
    VALIDATION_ACCURACY_SPEECH.append(results['accuracy'])
    VALIDATION_LOSS_SPEECH.append(results['loss'])
	
    tf.keras.backend.clear_session()
	
    fold_var += 1
     

Epoch 1/80
936/936 [==============================] - 27s 28ms/step - loss: 1.7154 - accuracy: 0.3632 - val_loss: 1.2558 - val_accuracy: 0.5481
Epoch 2/80
936/936 [==============================] - 26s 28ms/step - loss: 1.3874 - accuracy: 0.4757 - val_loss: 1.1045 - val_accuracy: 0.6175
Epoch 3/80
936/936 [==============================] - 26s 28ms/step - loss: 1.2390 - accuracy: 0.5334 - val_loss: 1.0654 - val_accuracy: 0.6261
Epoch 4/80
936/936 [==============================] - 26s 28ms/step - loss: 1.1346 - accuracy: 0.5820 - val_loss: 1.0379 - val_accuracy: 0.6293
Epoch 5/80
936/936 [==============================] - 26s 28ms/step - loss: 1.0543 - accuracy: 0.6042 - val_loss: 1.0018 - val_accuracy: 0.6464
Epoch 6/80
936/936 [==============================] - 26s 28ms/step - loss: 0.9768 - accuracy: 0.6328 - val_loss: 0.9783 - val_accuracy: 0.6528
Epoch 7/80
936/936 [==============================] - 26s 27ms/step - loss: 0.9252 - accuracy: 0.6638 - val_loss: 0.9801 - val_accuracy:

In [70]:
#Fairness
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
save_dir = os.path.abspath(DIR)+'/cremad_40m60f_earlyfusion_PCA_n0.95_model/'
final_model = multi_modal_NN()
final_model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=["accuracy"])
final_model.load_weights(save_dir + "model_1.h5")  # Adjust to load the best model if necessary

test_results = final_model.evaluate(concatenated_test_data, y_test)
test_results = dict(zip(final_model.metrics_names, test_results))

print("Test set evaluation:", test_results)

# Gender bias analysis
metrics_by_gender = {}
for gender in np.unique(gender_test):
    gender_indices = np.where(gender_test == gender)
    X_gender_test = concatenated_test_data[gender_indices]
    y_gender_test = y_test[gender_indices]
    y_pred = np.argmax(final_model.predict(X_gender_test), axis=1)

    accuracy = np.mean(y_pred == y_gender_test)
    precision = precision_score(y_gender_test, y_pred, average='macro')
    recall = recall_score(y_gender_test, y_pred, average='macro')
    f1 = f1_score(y_gender_test, y_pred, average='macro')

    cm = confusion_matrix(y_gender_test, y_pred)
    tn = np.sum(cm) - (np.sum(cm, axis=0) + np.sum(cm, axis=1) - np.diag(cm))
    fp = np.sum(cm, axis=0) - np.diag(cm)
    fn = np.sum(cm, axis=1) - np.diag(cm)
    tp = np.diag(cm)
    tpr = tp / (tp + fn)  # True Positive Rate
    fpr = fp / (fp + tn)  # False Positive Rate
    fnr = fn / (fn + tp)  # False Negative Rate
    precision_value = tp / (tp + fp)

    demographic_parity_dict = {}
    total_count  = len(y_pred)
    for emotion_idx in np.unique(y_pred):
        #emotion_idx = int(emotion_idx)
        count_emotion = np.count_nonzero(y_pred == emotion_idx)
        demographic_parity = count_emotion / total_count 
        emotion_name = emotions_en[emotion_idx]  # Get the emotion name from the index
        demographic_parity_dict[emotion_name] = demographic_parity

    
    metrics_by_gender[gender] = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'tpr': tpr,
        'fpr_value': fpr,
        'fnr_value': fnr,
        'precision_value': precision_value,
        'demographic_parity': demographic_parity_dict
    }
    

for gender in metrics_by_gender:
    #metrics_by_gender[gender]['demographic_parity'] = metrics_by_gender[gender]['positive_rate']
    metrics_by_gender[gender]['equalized_odds'] = (metrics_by_gender[gender]['tpr'], metrics_by_gender[gender]['fpr_value'])
    metrics_by_gender[gender]['equal_opportunity'] = metrics_by_gender[gender]['tpr']
    metrics_by_gender[gender]['predictive_parity'] = metrics_by_gender[gender]['precision_value']

47/47 [==============================] - 1s 7ms/step - loss: 1.0619 - accuracy: 0.6537
Test set evaluation: {'loss': 1.061920166015625, 'accuracy': 0.6536651253700256}
25/25 [==============================] - 0s 7ms/step


In [71]:
header = ['gender', 'accuracy', 'precision', 'recall', 'f1', 'tpr', 'fpr_value', 'fnr_value', 'precision_value', 'demographic_parity']
import csv
file_path = '/var/scratch/mpa326/CREMA-D_code/FairnessResults/fairness_analysis_cremad_40m60f_earlyfusion_model.csv'

# Ensure the directory exists
os.makedirs(os.path.dirname(file_path), exist_ok=True)
# Write the dictionary to a CSV file
with open(file_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(header)
    
    for gender, metrics in metrics_by_gender.items():
        row = [gender]
        for key in header[1:]:  # Exclude 'gender' from header keys
            value = metrics.get(key, None)
            # Convert dictionaries to string for CSV export
            if isinstance(value, dict):
                value = str(value)
            row.append(value)
        writer.writerow(row)
    overall_row = ['overall']
    for key in header[1:]:
        # Add overall accuracy and loss, leave other columns empty (None)
        if key in test_results:
            overall_row.append(test_results[key])
        else:
            overall_row.append(None)

In [ ]:
#Mid Fusion

In [74]:
from tensorflow.keras.models import load_model
audio_indice = 1
save_dir = os.path.abspath(DIR)+'/cremad_50m50f_audio_model/'
model_audio = model_alexnet()
model_audio.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=["accuracy"])
model_audio.load_weights(save_dir +"model_"+str(audio_indice)+".h5")

In [75]:
save_dir = os.path.abspath(DIR)+'/cremad_50m50f_video_model/'
video_indice = 1
model_video = create__LRCN_with_STN()
model_video.compile(loss='sparese_categorical_crossentropy', optimizer='Adam', metrics=["accuracy"])
model_video.load_weights(save_dir+"model_"+str(video_indice)+".h5")

Building Localization Network with input shape: (None, 48, 48, 1)
Building Bilinear Interpolation Layer with input shape: ((None, 48, 48, 1), (None, 2, 3))


In [77]:
features_video_model=Model(inputs=model_video.input, outputs=model_video.get_layer('time_distributed_14').output)

In [78]:
features_video=features_video_model.predict(X_train_50m_50f)

147/147 [==============================] - 21s 142ms/step


In [79]:
features_video.shape

(4678, 20, 1024)

In [80]:
X_train_flattened = np.reshape(features_video, (4678, -1))

In [81]:
features_video_test = features_video_model.predict(X_test)

47/47 [==============================] - 7s 137ms/step


In [82]:
X_test_flattened = np.reshape(features_video_test, (1487, -1))

In [84]:
features_audio_model=Model(inputs=model_audio.input, outputs=model_audio.get_layer('flatten_11').output)

In [85]:
features_audio=features_audio_model.predict(X_train_aud_50m_50f)

147/147 [==============================] - 44s 298ms/step


In [86]:
features_audio_test = features_audio_model.predict(X_test_aud)

47/47 [==============================] - 14s 289ms/step


In [87]:
from sklearn.preprocessing import StandardScaler
video_scaler = StandardScaler().fit(X_train_flattened)
video_data_standardized = video_scaler.fit_transform(X_train_flattened)
video_data_standardized_test = video_scaler.fit_transform(X_test_flattened)

In [88]:
audio_scaler = StandardScaler().fit(features_audio)
audio_data_standardized = audio_scaler.fit_transform(features_audio)
audio_data_standardized_test = audio_scaler.fit_transform(features_audio_test)

In [89]:
from sklearn.decomposition import PCA
pca_video = PCA(n_components=0.95).fit(video_data_standardized)
video_data_pca = pca_video.fit_transform(video_data_standardized)
video_data_pca_test = pca_video.transform(video_data_standardized_test)

In [90]:
video_data_pca.shape

(4678, 1950)

In [91]:
pca_audio = PCA(n_components=0.95).fit(audio_data_standardized)
audio_data_pca = pca_audio.fit_transform(audio_data_standardized)
audio_data_pca_test = pca_audio.transform(audio_data_standardized_test)

In [92]:
audio_data_pca.shape

(4678, 1846)

In [93]:
concatenated_train_data = np.concatenate((video_data_pca, audio_data_pca), axis=1)

In [94]:
concatenated_test_data = np.concatenate((video_data_pca_test, audio_data_pca_test), axis=1)

In [95]:
concatenated_train_data.shape

(4678, 3796)

In [96]:
def multi_modal_NN():
  model=Sequential()
  model.add(Input(shape=(3796)))
  
  model.add(Dense(2096))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0.4))

  model.add(Dense(1024))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0.4))

  model.add(Dense(8))
  model.add(BatchNormalization())
  model.add(Activation('softmax'))
  return model

In [103]:
VALIDATION_ACCURACY_SPEECH = []
VALIDATION_LOSS_SPEECH = []

save_dir = os.path.abspath(DIR) + '/cremad_40m60f_intermediatefusion_PCA_n0.95_model/'
fold_var = 1

for train_idx, val_idx in kfold.split(concatenated_train_data, y_train_40m_60f):
    model=multi_modal_NN()

    model.compile(loss='sparse_categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])

    early_stopping_callback = EarlyStopping(monitor = 'val_accuracy', patience = 15, restore_best_weights = True)

    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(save_dir + get_model_name(fold_var), monitor='val_accuracy',save_best_only=True, mode='max')
# Start training the model.
    LRCN_model_training_history = model.fit(      x = concatenated_train_data[train_idx],
                                                  y = y_train_40m_60f[train_idx],
                                                  validation_data=(concatenated_train_data[val_idx], y_train_40m_60f[val_idx]),
                                                  epochs = 80,
                                                  batch_size = 4,
                                                  shuffle = True,
                                                  callbacks = [checkpoint_cb, early_stopping_callback])
    model.load_weights(save_dir + "model_" + str(fold_var) + ".h5")
	
    results = model.evaluate(concatenated_test_data, y_test)
    results = dict(zip(model.metrics_names, results))
	
    VALIDATION_ACCURACY_SPEECH.append(results['accuracy'])
    VALIDATION_LOSS_SPEECH.append(results['loss'])
	
    tf.keras.backend.clear_session()
	
    fold_var += 1
     

Epoch 1/80
936/936 [==============================] - 38s 39ms/step - loss: 1.5518 - accuracy: 0.4372 - val_loss: 1.0264 - val_accuracy: 0.6485
Epoch 2/80
936/936 [==============================] - 34s 36ms/step - loss: 1.2011 - accuracy: 0.5508 - val_loss: 0.9465 - val_accuracy: 0.6677
Epoch 3/80
936/936 [==============================] - 34s 36ms/step - loss: 1.0843 - accuracy: 0.5989 - val_loss: 0.9409 - val_accuracy: 0.6934
Epoch 4/80
936/936 [==============================] - 33s 36ms/step - loss: 0.9959 - accuracy: 0.6368 - val_loss: 1.0241 - val_accuracy: 0.6464
Epoch 5/80
936/936 [==============================] - 33s 35ms/step - loss: 0.9652 - accuracy: 0.6416 - val_loss: 0.9713 - val_accuracy: 0.6592
Epoch 6/80
936/936 [==============================] - 33s 36ms/step - loss: 0.8799 - accuracy: 0.6852 - val_loss: 0.9721 - val_accuracy: 0.6560
Epoch 7/80
936/936 [==============================] - 34s 36ms/step - loss: 0.8262 - accuracy: 0.7028 - val_loss: 0.9917 - val_accuracy:

In [97]:
#Fairness
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
save_dir = os.path.abspath(DIR)+'/cremad_50m50f_intermediatefusion_PCA_n0.95_model/'
final_model = multi_modal_NN()
final_model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=["accuracy"])
final_model.load_weights(save_dir + "model_3.h5")  # Adjust to load the best model if necessary

test_results = final_model.evaluate(concatenated_test_data, y_test)
test_results = dict(zip(final_model.metrics_names, test_results))

print("Test set evaluation:", test_results)

# Gender bias analysis
metrics_by_gender = {}
for gender in np.unique(gender_test):
    gender_indices = np.where(gender_test == gender)
    X_gender_test = concatenated_test_data[gender_indices]
    y_gender_test = y_test[gender_indices]
    y_pred = np.argmax(final_model.predict(X_gender_test), axis=1)

    accuracy = np.mean(y_pred == y_gender_test)
    precision = precision_score(y_gender_test, y_pred, average='macro')
    recall = recall_score(y_gender_test, y_pred, average='macro')
    f1 = f1_score(y_gender_test, y_pred, average='macro')

    cm = confusion_matrix(y_gender_test, y_pred)
    tn = np.sum(cm) - (np.sum(cm, axis=0) + np.sum(cm, axis=1) - np.diag(cm))
    fp = np.sum(cm, axis=0) - np.diag(cm)
    fn = np.sum(cm, axis=1) - np.diag(cm)
    tp = np.diag(cm)
    tpr = tp / (tp + fn)  # True Positive Rate
    fpr = fp / (fp + tn)  # False Positive Rate
    fnr = fn / (fn + tp)  # False Negative Rate
    precision_value = tp / (tp + fp)

    demographic_parity_dict = {}
    total_count  = len(y_pred)
    for emotion_idx in np.unique(y_pred):
        #emotion_idx = int(emotion_idx)
        count_emotion = np.count_nonzero(y_pred == emotion_idx)
        demographic_parity = count_emotion / total_count 
        emotion_name = emotions_en[emotion_idx]  # Get the emotion name from the index
        demographic_parity_dict[emotion_name] = demographic_parity

    
    metrics_by_gender[gender] = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'tpr': tpr,
        'fpr_value': fpr,
        'fnr_value': fnr,
        'precision_value': precision_value,
        'demographic_parity': demographic_parity_dict
    }
    

#Additional fairness metrics

# for gender in np.unique(gender_test):
#     gender_indices = np.where(gender_test == gender)
#     print(gender)
#     predicted_label_1 = predicted_labels[gender_indices]
#     total_count = len(predicted_label_1)
    

for gender in metrics_by_gender:
    #metrics_by_gender[gender]['demographic_parity'] = metrics_by_gender[gender]['positive_rate']
    metrics_by_gender[gender]['equalized_odds'] = (metrics_by_gender[gender]['tpr'], metrics_by_gender[gender]['fpr_value'])
    metrics_by_gender[gender]['equal_opportunity'] = metrics_by_gender[gender]['tpr']
    metrics_by_gender[gender]['predictive_parity'] = metrics_by_gender[gender]['precision_value']



47/47 [==============================] - 1s 13ms/step - loss: 0.9729 - accuracy: 0.6698
Test set evaluation: {'loss': 0.9728664755821228, 'accuracy': 0.6698049902915955}
25/25 [==============================] - 0s 13ms/step


In [98]:
header = ['gender', 'accuracy', 'precision', 'recall', 'f1', 'tpr', 'fpr_value', 'fnr_value', 'precision_value', 'demographic_parity']
import csv
file_path = '/var/scratch/mpa326/CREMA-D_code/FairnessResults/fairness_analysis_cremad_50m50f_intermediatefusion_model.csv'

# Ensure the directory exists
os.makedirs(os.path.dirname(file_path), exist_ok=True)
# Write the dictionary to a CSV file
with open(file_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(header)
    
    for gender, metrics in metrics_by_gender.items():
        row = [gender]
        for key in header[1:]:  # Exclude 'gender' from header keys
            value = metrics.get(key, None)
            # Convert dictionaries to string for CSV export
            if isinstance(value, dict):
                value = str(value)
            row.append(value)
        writer.writerow(row)
    overall_row = ['overall']
    for key in header[1:]:
        # Add overall accuracy and loss, leave other columns empty (None)
        if key in test_results:
            overall_row.append(test_results[key])
        else:
            overall_row.append(None)

In [ ]:
#Unimodal Fairness

In [46]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
save_dir = os.path.abspath(DIR)+'/cremad_50m50f_video_model/'
final_model = create__LRCN_with_STN()
final_model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=["accuracy"])
final_model.load_weights(save_dir + "model_1.h5")  # Adjust to load the best model if necessary

test_results = final_model.evaluate(X_test, y_test)
test_results = dict(zip(final_model.metrics_names, test_results))

print("Test set evaluation:", test_results)

# Gender bias analysis
metrics_by_gender = {}
for gender in np.unique(gender_test):
    gender_indices = np.where(gender_test == gender)
    X_gender_test = X_test[gender_indices]
    y_gender_test = y_test[gender_indices]
    y_pred = np.argmax(final_model.predict(X_gender_test), axis=1)

    accuracy = np.mean(y_pred == y_gender_test)
    precision = precision_score(y_gender_test, y_pred, average='macro')
    recall = recall_score(y_gender_test, y_pred, average='macro')
    f1 = f1_score(y_gender_test, y_pred, average='macro')

    cm = confusion_matrix(y_gender_test, y_pred)
    tn = np.sum(cm) - (np.sum(cm, axis=0) + np.sum(cm, axis=1) - np.diag(cm))
    fp = np.sum(cm, axis=0) - np.diag(cm)
    fn = np.sum(cm, axis=1) - np.diag(cm)
    tp = np.diag(cm)
    tpr = tp / (tp + fn)  # True Positive Rate
    fpr = fp / (fp + tn)  # False Positive Rate
    fnr = fn / (fn + tp)  # False Negative Rate
    precision_value = tp / (tp + fp)

    demographic_parity_dict = {}
    total_count  = len(y_pred)
    for emotion_idx in np.unique(y_pred):
        #emotion_idx = int(emotion_idx)
        count_emotion = np.count_nonzero(y_pred == emotion_idx)
        demographic_parity = count_emotion / total_count 
        emotion_name = emotions_en[emotion_idx]  # Get the emotion name from the index
        demographic_parity_dict[emotion_name] = demographic_parity

    
    metrics_by_gender[gender] = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'tpr': tpr,
        'fpr_value': fpr,
        'fnr_value': fnr,
        'precision_value': precision_value,
        'demographic_parity': demographic_parity_dict
    }
    

#Additional fairness metrics

# for gender in np.unique(gender_test):
#     gender_indices = np.where(gender_test == gender)
#     print(gender)
#     predicted_label_1 = predicted_labels[gender_indices]
#     total_count = len(predicted_label_1)
    

for gender in metrics_by_gender:
    #metrics_by_gender[gender]['demographic_parity'] = metrics_by_gender[gender]['positive_rate']
    metrics_by_gender[gender]['equalized_odds'] = (metrics_by_gender[gender]['tpr'], metrics_by_gender[gender]['fpr_value'])
    metrics_by_gender[gender]['equal_opportunity'] = metrics_by_gender[gender]['tpr']
    metrics_by_gender[gender]['predictive_parity'] = metrics_by_gender[gender]['precision_value']



Building Localization Network with input shape: (None, 48, 48, 1)
Building Bilinear Interpolation Layer with input shape: ((None, 48, 48, 1), (None, 2, 3))
47/47 [==============================] - 9s 151ms/step - loss: 1.6351 - accuracy: 0.5898
Test set evaluation: {'loss': 1.635090708732605, 'accuracy': 0.5897780656814575}
25/25 [==============================] - 4s 147ms/step


In [47]:
header = ['gender', 'accuracy', 'precision', 'recall', 'f1', 'tpr', 'fpr_value', 'fnr_value', 'precision_value', 'demographic_parity']
import csv
file_path = '/var/scratch/mpa326/CREMA-D_code/FairnessResults/fairness_analysis_cremad_50m50f_video_model.csv'

# Ensure the directory exists
os.makedirs(os.path.dirname(file_path), exist_ok=True)
# Write the dictionary to a CSV file
with open(file_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(header)
    
    for gender, metrics in metrics_by_gender.items():
        row = [gender]
        for key in header[1:]:  # Exclude 'gender' from header keys
            value = metrics.get(key, None)
            # Convert dictionaries to string for CSV export
            if isinstance(value, dict):
                value = str(value)
            row.append(value)
        writer.writerow(row)
    overall_row = ['overall']
    for key in header[1:]:
        # Add overall accuracy and loss, leave other columns empty (None)
        if key in test_results:
            overall_row.append(test_results[key])
        else:
            overall_row.append(None)